In [1]:
import sys
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install pyyaml

In [2]:
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("DB_NAME")]
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))
# counter for figures
counter = 0

In [3]:
# Look through errors.
error_types = {}
for name in db.list_collection_names():
    if "error" in name:
        for error in db[name].find({"timestamp":{"$gt": 1534446003520}}):
            #print(name)
            #print(json.loads(error["error"]))
            stack_trace = json.loads(error["error"])["stackTrace"]
            if stack_trace not in error_types:
                error_types[stack_trace] = 1
            else:
                error_types[stack_trace] += 1
for stack in error_types:
    print("STACKTRACE WITH FREQ " +str(error_types[stack]))
    print(stack)
    print("\n\n")

STACKTRACE WITH FREQ 1
An uncaught Exception occurred on "main" thread.
android.content.res.Resources$NotFoundException: Resource ID #0xffffffec
	at android.content.res.ResourcesImpl.getValue(ResourcesImpl.java:202)
	at android.content.res.Resources.loadXmlResourceParser(Resources.java:2968)
	at android.content.res.Resources.getAnimation(Resources.java:2008)
	at android.animation.AnimatorInflater.loadAnimator(AnimatorInflater.java:125)
	at android.animation.AnimatorInflater.loadAnimator(AnimatorInflater.java:106)
	at android.animation.AnimatorInflater.loadAnimator(AnimatorInflater.java:91)
	at android.app.FragmentManagerImpl.loadAnimator(FragmentManager.java:827)
	at android.app.FragmentManagerImpl.moveToState(FragmentManager.java:1065)
	at android.app.FragmentManagerImpl.removeFragment(FragmentManager.java:1280)
	at android.app.BackStackRecord.run(BackStackRecord.java:766)
	at android.app.FragmentManagerImpl.execPendingActions(FragmentManager.java:1582)
	at android.app.FragmentManager

In [5]:
# Unfortunately, we're still getting those erroneous logs of huge time spent on "" domain since some updates aren't fully reflected yet. Let's delete them again.
done = False
for account in accounts:
    if not done:
        for user in account["android"]:
            if not done:
                print("USER ---- " + user)
                if len([s for s in db[user + "_sessions"].find({"duration": {"$gt": 60 ** 2 * 24 * 7}})]) > 0:
                    print("Num of sessions: " + str(len([s for s in db[user+"_sessions"].find()])))
                    print("Num of bad sessions: " + str(len([s for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})])))
                    print("VERSIONS" + str({s['version'] for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})}))
                    db[user + "_sessions"].delete_many({"domain": "","duration": {"$gt": 60 ** 2 * 24}})
                    print("After delete: " + str(len([s for s in db[user+"_sessions"].find()])))



USER ---- 9cb8c59eabbb60ad6447345c
USER ---- b219926d16e6e46fa426cafd
USER ---- U1532437453047928
Num of sessions: 1998
Num of bad sessions: 2
VERSIONS{38}
After delete: 1996
USER ---- 010cf6179f52b3d62184ee17
Num of sessions: 4816
Num of bad sessions: 1
VERSIONS{38}
After delete: 4815
USER ---- 0b0d1da5c036cb9637aff5d5
USER ---- f6c5e87bbe828ffc4278d4b7
USER ---- 38a4b3e06ead18a13442f5a7
USER ---- f463f9d5191a4bc1d1374298
USER ---- c4f80e629327886902828290
USER ---- 7f2cc2756b7c8c658a86ce6c
USER ---- 6351db97a5f1304f5dd86f0f
USER ---- 619164f4d106d7c53658931c
USER ---- 640c8c27aad1006852979eb5
USER ---- 8efcdb3217ad19868c86c2a4
USER ---- bdc472cb840f151f72f88ec3
USER ---- ec0dbd76f4093d26176ba153
USER ---- 855de136c8e71443d24b081b
USER ---- c8dc7f9f2901e94fd90b9a05
USER ---- 3d9e01f13145754f3ee22cdb
Num of sessions: 3629
Num of bad sessions: 1
VERSIONS{38}
After delete: 3628
USER ---- 0cb53f687a648364e24cb0da
USER ---- c9991373fd8cd5ce69292511
USER ---- 87d22aeadda00f729112c77a
USER -

Num of sessions: 4320
Num of bad sessions: 2
VERSIONS{38}
After delete: 4319
USER ---- 1cd8082b142d956d6f315964
USER ---- bc8004f2d5954bac43740d66
USER ---- 8b9ce7d429640799fdd936a2
USER ---- f425b912351840fc002aa901
USER ---- b0bb9bbfa279f214f4a2052d
USER ---- a8887a6e2070ced6342a832d
USER ---- a3c75afed38512390aca34b6
USER ---- 6377acce2a54e6971d4702f6
USER ---- 8b1a629b98c777872a7bfdbd
Num of sessions: 983
Num of bad sessions: 1
VERSIONS{38}
After delete: 982
USER ---- 708dbfaebbbba9065074286d
USER ---- dc3b87df83232926790040ec
USER ---- 8986fee529fbb0d9d7088af3
USER ---- c6c086cbeedcb01530ed214a
USER ---- 42a5c2bd41f829b48818af38
USER ---- 9cc8c74f9d40ea67c67843cc
USER ---- ea6c441c96df6073aebea4c5
USER ---- 42031f51af2f72490147713e
USER ---- a3653d7b076174841ae96fa4
USER ---- d36fb3141afddf778a0ea5cd
USER ---- eaa4533e641359d74b28fa16
USER ---- c8fe161c8cc1a2ad3b9ae057
USER ---- 09927d362c1167b3ea2c42d7
USER ---- eaf9d14ff7af67ab677da20b
USER ---- b90811afb7617eba4d7c2064
USER ---